In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Yelp API Key
gkey = 'fxuoEjmZHd5kA1x20u4z2waSsXfTm49pWr2F6ac-htERJ0haCqXH949tlmDYt9lhy4MC6nBJjt2llMwhQJfxbwX69QlFkI221WkMDZTn7NMQBaXRrWGBFzCrg4-9XXYx'


In [2]:
from etl_clean_ip import zip_list

/Users/monicamalarcon/Dropbox/School/UM_Bootcamp/Work_Bootcamp/13 - ETL-Project/ETL-project/ETL-Project/Data/etl_clean_ip.py:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  top30_disparity['zip_code'] = top30_disparity['zip_code'].astype(int)
/Users/monicamalarcon/Dropbox/School/UM_Bootcamp/Work_Bootcamp/13 - ETL-Project/ETL-project/ETL-Project/Data/etl_clean_ip.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  top30_disparity['Avg_income'] = top30_disparity['Avg_income'].round(3)


In [3]:
#set up base url
# base url
base_url = "https://api.yelp.com/v3/businesses/search"

# set up api_key dictionary
headers = {
        'Authorization': 'Bearer %s' % gkey,
    }

# set up a parameters dictionary
def s_params(target_loc,target_key,target_type):
    params = {
    "location": target_loc,
    "term": target_key,
    "categories": target_type,
}
    return params

# run a request using our params dictionary
def s_query(base_url,headers,params):
    response = requests.request('GET', base_url, headers=headers, params=params).json()
    return response

In [4]:
# Import zip list
zip_list 


#cities_pd.shape[0] #use .shape[0] to count the number of rows

[33142,
 33127,
 33147,
 33125,
 33136,
 33012,
 33167,
 33126,
 33150,
 33032,
 33130,
 33162,
 33034,
 33055,
 33056,
 33174,
 33172,
 33033,
 33168,
 33190,
 33169,
 33181,
 33030,
 33014,
 33193,
 33016,
 33161,
 33184,
 33194,
 33035]

In [13]:
# create empty lists to hold yelp data
names = []
addresses = []
lats = []
lons = []
ratings = []
price = []

#define Yelp function to get restaurant data
def get_restaurants(loc):
    
    #these are the additional params
    add_params = { "limit" : 10, "radius" : 40000}

    #these are the original params
    params = s_params(loc,"restaurants","restaurants, All")

    #add additional params to our params from before
    s1_params = {**params,**add_params}

    s1_results = s_query(base_url,headers,s1_params)
    
    if (s1_results):
    
        name = s1_results["businesses"][0]["name"]
        address = s1_results["businesses"][0]["location"]["display_address"]
        lat = s1_results["businesses"][0]["coordinates"]["latitude"]
        lon = s1_results["businesses"][0]["coordinates"]["longitude"]
        rating = s1_results["businesses"][0]["rating"]
        price = s1_results["businesses"][0]["price"]
        output = [name,address,lat,lon,rating,price] 
    else:
        output = ["NaN","NaN","NaN","NaN","NaN","NaN"]
    
    
    return output


In [15]:
# Loop through the zipcodes in zip_list and append restaurant info to lists above

print("wait...")
print("Beginning Data Retrieval")
print("-----------------------------------")

for zipc in zip_list:
    loc = zipc
    try:
        output = get_restaurants(loc)
        print(output)
        names.append(output[0])
        addresses.append(output[1])
        lats.append(output[2])
        lons.append(output[3])
        ratings.append(output[4])
        price.append(output[5])
        
# make exception for key errors        
    except (KeyError, IndexError):
        print(f'Key missing')
        output = ["NaN","NaN","NaN","NaN","NaN","NaN"]
        names.append(output[0])
        addresses.append(output[1])
        lats.append(output[2])
        lons.append(output[3])
        ratings.append(output[4])
        price.append(output[5])

print("-----------------------------------")
print("Data Retrieval Complete")

    
print("done!")

wait...
Beginning Data Retrieval
-----------------------------------
['Hometown Bar-B-Que', ['1200 NW 22nd St', 'Ste 100', 'Miami, FL 33142'], 25.797445333, -80.215986833, 4.5, '$$']
['KYU', ['251 NW 25th St', 'Miami, FL 33127'], 25.80089, -80.20017, 4.5, '$$$']
['Sins Gastrobar', ['9835 NE 2nd Ave', 'Miami Shores, FL 33138'], 25.86562, -80.19331, 4.0, '$$']
["Old's Havana Cuban Bar & Cocina", ['1442 SW 8th St', 'Miami, FL 33135'], 25.7655942148975, -80.2192383373502, 4.5, '$$']
["Old's Havana Cuban Bar & Cocina", ['1442 SW 8th St', 'Miami, FL 33135'], 25.7655942148975, -80.2192383373502, 4.5, '$$']
Key missing
['Sins Gastrobar', ['9835 NE 2nd Ave', 'Miami Shores, FL 33138'], 25.86562, -80.19331, 4.0, '$$']
['Amor di Pasta - Blue Lagoon', ['536 NW 57th Ave', 'Miami, FL 33126'], 25.776908, -80.288655, 4.5, '$$']
['Sins Gastrobar', ['9835 NE 2nd Ave', 'Miami Shores, FL 33138'], 25.86562, -80.19331, 4.0, '$$']
Key missing
["Old's Havana Cuban Bar & Cocina", ['1442 SW 8th St', 'Miami, FL 3

In [19]:
# Add columns for lat, lng, airport name, airport address, airport rating
restaurants_df = pd.DataFrame({"Name": names,"Location": addresses,"Rating": ratings,"Price": price})
restaurants_df.dropna(how="any")
restaurants_df

,Name,Location,Rating,Price
0,Hometown Bar-B-Que,"[1200 NW 22nd St, Ste 100, Miami, FL 33142]",4.5,$$
1,KYU,"[251 NW 25th St, Miami, FL 33127]",4.5,$$$
2,Sins Gastrobar,"[9835 NE 2nd Ave, Miami Shores, FL 33138]",4,$$
3,Old's Havana Cuban Bar & Cocina,"[1442 SW 8th St, Miami, FL 33135]",4.5,$$
4,Old's Havana Cuban Bar & Cocina,"[1442 SW 8th St, Miami, FL 33135]",4.5,$$
5,NaN,NaN,NaN,NaN
6,Sins Gastrobar,"[9835 NE 2nd Ave, Miami Shores, FL 33138]",4,$$
7,Amor di Pasta - Blue Lagoon,"[536 NW 57th Ave, Miami, FL 33126]",4.5,$$
8,Sins Gastrobar,"[9835 NE 2nd Ave, Miami Shores, FL 33138]",4,$$
9,NaN,NaN,NaN,NaN


In [ ]:
print(f"The data frame contains {len(restaurants_df['Name'])} unique restaurants.")

      
restaurants_df

In [ ]:
# Save Data to csv
#cities_pd.to_csv("Airport_Output.csv")

# Visualize to confirm airport data appears
#cities_pd.head(10)